In [46]:
from ams_paq_utilities import *
from ams_utilities import *
import scipy
import matplotlib.pyplot as plt
%matplotlib notebook
import pyabf
import paq2py
import numpy as np
import pandas as pd
import statistics

In [23]:
# Import data from spreadsheet 
df_all = GS_to_df('https://docs.google.com/spreadsheets/d/1ziOx80em0ZhmMmSjKePYbOq3K6sHcOapfHjy9S4oDbk/edit#gid=662691853')
df_all.head()

,Mouse#: Cell,AS016: Cell01,AS016: Cell02,AS029: Cell01,AS030: Cell01,AS035: Cell01,AS036: Cell01,AS038: Cell01,AS039: Cell01,AS039: Cell02,...,AS114: Cell01,AS114: Cell02,AS114: Cell03,AS115: Cell01,AS116: Cell01,AS116: Cell02,AS117: Cell04,AS132: Cell03,AS143: Cell03,AS152: Cell01
0,Slice_Orientation,H,H,H,H,H,C,H,H,H,...,C,C,C,C,C,C,C,C,C,C
1,Axon_node_total,6,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,5,6,6,0
2,Axon_length,1620.49,0,28.4,0,0,0,0,0,0,...,118.66,0,0,0,0,0,355.68,1137.77,629.87,123.85
3,Axon_SA,10859.8,0,104.62,0,0,0,0,0,0,...,454.33,0,0,0,0,0,981.42,5585.99,2131.56,189.15
4,Axon_length_to_SA_ratio,0.1492191385,NaN,0.2714586121,NaN,NaN,NaN,NaN,NaN,NaN,...,0.2611757973,NaN,NaN,NaN,NaN,NaN,0.3624136455,0.203682785,0.2954971945,0.6547713455


In [33]:
# Extents and oreintations df
extents_all=df_all.iloc[61,:]
orientations_all=df_all.iloc[0,:]
df=pd.concat([extents_all,orientations_all],1)
df=df.drop('Mouse#: Cell',axis=0)
df

,61,0
AS016: Cell01,400,H
AS016: Cell02,130,H
AS029: Cell01,190,H
AS030: Cell01,420,H
AS035: Cell01,110,H
AS036: Cell01,460,C
AS038: Cell01,230,H
AS039: Cell01,300,H
AS039: Cell02,70,H
AS040: Cell01,80,H


In [62]:
# Make lists of horizontal and coronal extents
h_extents=[]
c_extents=[]
extents_str=list(df.iloc[:,0])
extents=[]
for i in extents_str:
    i=int(i)
    extents.append(i)

for i in range(0,len(extents)):
    if df.iat[i,1]=='H':
        h_extents.append(extents[i])
    elif df.iat[i,1]=='C':
        c_extents.append(extents[i])
    else:
        print('Error')

[400, 130, 190, 420, 110, 230, 300, 70, 80, 200, 170, 180, 190, 150, 310, 310, 220, 160, 470, 290]


In [70]:
#Are the data normally distributed? p>0.05 suggests yes. Kolmogorov-Smirnov test
h_mean=statistics.mean(h_extents)
h_std=statistics.stdev(h_extents)
print('Horizontal - ' + str(scipy.stats.kstest(h_extents, 'norm', args=(h_mean,h_std))))

c_mean=statistics.mean(c_extents)
c_std=statistics.stdev(c_extents)
print('Coronal - ' + str(scipy.stats.kstest(c_extents, 'norm', args=(c_mean,c_std))))

Horizontal - KstestResult(statistic=0.15222950932149093, pvalue=0.7191047745870478)
Coronal - KstestResult(statistic=0.13880055258580526, pvalue=0.43919037128085386)


In [72]:
#T-test, since normal distribution can be assumed
scipy.stats.ttest_ind(h_extents, c_extents, equal_var=False)

Ttest_indResult(statistic=-0.45292175335176177, pvalue=0.6531534278339806)

In [85]:
#D'Agostino normality test
print('DAgostino\nHorizontal - '+str(scipy.stats.normaltest(h_extents)))
print('Coronal - '+str(scipy.stats.normaltest(c_extents))+'\n')

#Shapiro-Wilk normality test
print('Shapiro-Wilk\nHorizontal - '+str(scipy.stats.shapiro(h_extents)))
print('Coronal - '+str(scipy.stats.shapiro(c_extents)))

DAgostino
Horizontal - NormaltestResult(statistic=1.8610423530119573, pvalue=0.39434813182341677)
Coronal - NormaltestResult(statistic=6.354047666322032, pvalue=0.041709605325674935)

Shapiro-Wilk
Horizontal - (0.9409579038619995, 0.24997292459011078)
Coronal - (0.9368723034858704, 0.03672226518392563)


In [86]:
#Mann-Whitney U test
scipy.stats.mannwhitneyu(h_extents,c_extents)

MannwhitneyuResult(statistic=337.0, pvalue=0.29322849928389205)